# 01. LIF Neuron Dynamics

This notebook demonstrates the dynamics of the Leaky Integrate-and-Fire (LIF) neuron implemented in `pyngn`.

In [ ]:
import torch
import matplotlib.pyplot as plt
from pyngn.neuron import LIFLayer

%matplotlib inline

## Setup Neuron Layer

We create a layer of 5 neurons with different input currents.

In [ ]:
n_neurons = 5
dt = 1.0
tau_m = 20.0
v_th = 1.0

layer = LIFLayer(n_neurons=n_neurons, tau_m=tau_m, v_th=v_th, dt=dt)

# Input currents for each neuron
currents = torch.tensor([0.5, 0.9, 1.1, 1.5, 2.0])

## Simulation Loop

In [ ]:
duration = 200
voltage_trace = []
spike_trace = []

layer.reset_state()

for t in range(duration):
    # Constant current injection
    s = layer.forward(i_syn=currents)
    
    voltage_trace.append(layer.v.clone().numpy())
    spike_trace.append(s.clone().numpy())
    
import numpy as np
voltage_trace = np.array(voltage_trace)
spike_trace = np.array(spike_trace)

## Visualization

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(n_neurons):
    plt.plot(voltage_trace[:, i], label=f'Neuron {i} (I={currents[i].item():.1f})')

plt.axhline(v_th, color='k', linestyle='--', label='Threshold')
plt.title('Membrane Potential Dynamics')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.imshow(spike_trace.T, aspect='auto', cmap='Greys', interpolation='nearest')
plt.title('Spike Raster Plot')
plt.xlabel('Time (ms)')
plt.ylabel('Neuron Index')
plt.yticks(range(n_neurons))
plt.show()